In [9]:
import sys
import os
from pathlib import Path

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))



# notebooks/03_trading_simulation.ipynb

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from src.models.trading_engine import DynamicTradingEngine
from src.data_processor import DataProcessor
from src.models.expenditure_predictor import ExpenditurePredictor
import tensorflow as tf
from tensorflow.keras.models import Sequential



# Load data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')


# First, load and train models for each household

# Train prediction models
# household_predictors = {}
# for household in df['HouseholdID'].unique():
#     household_df = df[df['HouseholdID'] == household].copy()
#     X, y = processor.prepare_training_data(household_df)
    
#     # Split data
#     train_size = int(len(X) * 0.8)
#     X_train = X[:train_size]
#     y_train = y[:train_size]
    
#     # Train model
#     predictor = ExpenditurePredictor()
#     predictor.train(X_train, y_train)
#     household_predictors[household] = predictor
household_models = {}
for household in df['HouseholdID'].unique():
    model_path = f'../src/models/saved/model_{household}.keras'
    household_models[household] = tf.keras.models.load_model(model_path)

def simulate_future_trading(current_date, trained_models):
    """Simulate trading based on predicted values using pre-trained models"""
    future_trades = []
    
    for household in df['HouseholdID'].unique():
        # Get recent data for prediction
        recent_data = df[
            (df['HouseholdID'] == household) & 
            (df['Date'] <= current_date)
        ].tail(4)  # Last 4 weeks for prediction
        
        # Prepare input for prediction using the modified prepare_prediction_data method
        X_pred = processor.prepare_prediction_data(recent_data)
        
        # Use the pre-trained model for this household
        predictor = trained_models[household]
        predicted_expenditure = predictor.predict(X_pred)
        
        # Use last known values for usage and generation
        predicted_usage = recent_data['EnergyUsed'].mean()
        predicted_generation = recent_data['EnergyGeneratedFromRenewableSources'].mean()
        
        future_trades.append({
            'HouseholdID': household,
            'PredictedUsage': predicted_usage,
            'PredictedGeneration': predicted_generation,
            'PredictedExpenditure': predicted_expenditure[0]
        })
    
    return pd.DataFrame(future_trades)




# Initialize trading engine
engine = DynamicTradingEngine()

def get_season(date):
    month = pd.to_datetime(date).month
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

def simulate_weekly_trading(week_data, time_of_day=14, trained_models=household_models):
    # Use predicted values instead of historical data
    current_date = week_data['Date'].iloc[0]
    future_data = simulate_future_trading(current_date, trained_models)
    
    # Calculate predicted surplus/deficit
    future_data['Delta'] = future_data['PredictedGeneration'] - future_data['PredictedUsage']
    
    # Rest of the function remains similar but uses predicted values
    sellers = future_data[future_data['Delta'] > 0].copy()
    buyers = future_data[future_data['Delta'] < 0].copy()
    
    print(f"Time: {time_of_day}:00")
    print(f"Number of sellers: {len(sellers)}")
    print(f"Number of buyers: {len(buyers)}")
    
    if len(sellers) == 0 or len(buyers) == 0:
        return pd.DataFrame()
    
    # Get season for the trading day
    season = get_season(week_data['Date'].iloc[0])
    
    # Calculate dynamic price based on current market conditions
    current_market_price = engine.calculate_dynamic_price(
        sellers['Delta'].sum(),
        -buyers['Delta'].sum(),
        time_of_day,
        season
    )
    
    print(f"Current market price: ${current_market_price:.4f}/kWh")
    
    # Prepare sellers dataframe with more competitive rates
    sellers_df = pd.DataFrame({
        'householdID': sellers['HouseholdID'],
        'unitsToBeSold': sellers['Delta'],
        'sellingRate': current_market_price * 0.9  # More competitive selling rate
    })
    
    # Prepare buyers dataframe with higher buying rates
    buyers_df = pd.DataFrame({
        'householdID': buyers['HouseholdID'],
        'demand': -buyers['Delta'],
        'buyingRate': current_market_price * 1.1  # Higher buying rate to ensure trades
    })
    
    return engine.match_orders(sellers_df, buyers_df, time_of_day, season)

# Simulate trading for each week at different times
trading_times = [21]  # 9 AM, 2 PM, 7 PM
weekly_trades = []

for date in df['Date'].unique():
    week_data = df[df['Date'] == date]
    for time in trading_times:
        trades = simulate_weekly_trading(week_data, time)
        if not trades.empty:
            trades['Date'] = date
            trades['TimeOfDay'] = time
            weekly_trades.append(trades)
if not weekly_trades:
    print("No trades were generated during the simulation")
    trading_history = pd.DataFrame(columns=['buyer_id', 'seller_id', 'amount',
                                          'price_per_unit', 'total_cost',
                                          'timestamp', 'Date', 'TimeOfDay'])
else:
    trading_history = pd.concat(weekly_trades, ignore_index=True)

# Visualizations
# 1. Trading Volume Over Time with Time-of-Day Breakdown
fig1 = go.Figure()
for time in trading_times:
    time_data = trading_history[trading_history['TimeOfDay'] == time]
    daily_volume = time_data.groupby('Date')['amount'].sum()
    fig1.add_trace(go.Scatter(
        x=daily_volume.index,
        y=daily_volume.values,
        name=f'{time}:00',
        mode='lines'
    ))
fig1.update_layout(
    title='Trading Volume by Time of Day',
    xaxis_title='Date',
    yaxis_title='Energy Traded (kWh)'
)
fig1.show()

# 2. Price Trends by Time of Day
fig2 = go.Figure()
for time in trading_times:
    time_data = trading_history[trading_history['TimeOfDay'] == time]
    daily_price = time_data.groupby('Date')['price_per_unit'].mean()
    fig2.add_trace(go.Scatter(
        x=daily_price.index,
        y=daily_price.values,
        name=f'{time}:00',
        mode='lines'
    ))
fig2.update_layout(
    title='Average Trading Price by Time of Day',
    xaxis_title='Date',
    yaxis_title='Price ($/kWh)'
)
fig2.show()

# 3. Seasonal Trading Analysis
trading_history['Season'] = trading_history['Date'].apply(get_season)
seasonal_stats = trading_history.groupby('Season').agg({
    'amount': 'sum',
    'price_per_unit': 'mean',
    'total_cost': 'sum'
}).round(2)

print("\nSeasonal Trading Statistics:")
print(seasonal_stats)

# 4. Household Trading Summary
household_summary = pd.DataFrame({
    'Total_Sales': trading_history.groupby('seller_id')['total_cost'].sum(),
    'Total_Purchases': trading_history.groupby('buyer_id')['total_cost'].sum(),
    'Volume_Sold': trading_history.groupby('seller_id')['amount'].sum(),
    'Volume_Bought': trading_history.groupby('buyer_id')['amount'].sum()
}).fillna(0).round(2)

print("\nHousehold Trading Summary:")
print(household_summary)

# 5. Trading Network Visualization (Last Day)
def plot_trading_network(trades_sample):
    network_fig = go.Figure()
    
    for _, trade in trades_sample.iterrows():
        network_fig.add_trace(go.Scatter(
            x=[trade['seller_id'], trade['buyer_id']],
            y=[trade['amount'], trade['amount']],
            mode='lines+markers',
            name=f"${trade['total_cost']:.2f}",
            text=[f"Seller: {trade['seller_id']}<br>Price: ${trade['price_per_unit']:.3f}/kWh", 
                  f"Buyer: {trade['buyer_id']}<br>Amount: {trade['amount']} kWh"],
            hoverinfo='text'
        ))
    
    network_fig.update_layout(
        title=f'Energy Trading Network (Time: {trades_sample["TimeOfDay"].iloc[0]}:00)',
        showlegend=False
    )
    return network_fig

last_date = trading_history['Date'].max()
for time in trading_times:
    last_trades = trading_history[
        (trading_history['Date'] == last_date) & 
        (trading_history['TimeOfDay'] == time)
    ]
    if not last_trades.empty:
        network_fig = plot_trading_network(last_trades)
        network_fig.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 454ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
Time: 21:00
Number of sellers: 0
Number of buyers: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 410ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 481ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 524ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 489ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step
Time: 21:00
Number of sellers: 0
Number of buyers: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Time: 21:00
Number of sellers: 0
Number of buyers: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Time: 21:00
Number of sellers: 0
Number of buyers: 5
1/1 ━━━━━━━━━━━━━━


Seasonal Trading Statistics:
          amount  price_per_unit  total_cost
Season                                      
fall     7445.19            0.28     2027.46
spring  20098.40            0.32     6320.55
summer  18732.51            0.35     6691.01

Household Trading Summary:
    Total_Sales  Total_Purchases  Volume_Sold  Volume_Bought
H1        30.86          3652.57       101.33       11213.83
H2         0.00          6037.45         0.00       18662.80
H3     10445.48             6.62     32764.73          23.82
H4         0.00          5244.30         0.00       16053.58
H5      4562.68            98.08     13410.04         322.05


In [10]:
import sys
import os
from pathlib import Path
import tensorflow as tf

# Add parent directory to Python path
notebook_dir = Path(os.getcwd())
project_dir = notebook_dir.parent
sys.path.append(str(project_dir))
import pandas as pd

from src.data_processor import DataProcessor

# Load data
processor = DataProcessor()
df = processor.load_household_data('../data/sample_household_data.csv')

household_models = {}
for household in df['HouseholdID'].unique():
    model_path = f'../src/models/saved/model_{household}.keras'
    household_models[household] = tf.keras.models.load_model(model_path)

def predict_future_weeks(df, household_models, num_weeks=10):
    """Predict energy patterns and trading positions for next n weeks"""
    last_date = pd.to_datetime(df['Date'].max())
    future_dates = [last_date + pd.Timedelta(weeks=i+1) for i in range(num_weeks)]
    
    future_predictions = []
    
    for household in df['HouseholdID'].unique():
        # Get last 4 weeks of data for prediction
        recent_data = df[df['HouseholdID'] == household]
        X_pred = processor.prepare_prediction_data(recent_data)
        
        # Get predictions for this household
        predictor = household_models[household]
        
        # Use recent averages for baseline predictions
        base_usage = recent_data['EnergyUsed'].mean()
        base_generation = recent_data['EnergyGeneratedFromRenewableSources'].mean()
        
        for future_date in future_dates:
            # Adjust predictions based on season
            season = get_season(future_date)
            season_mult = {
                'summer': 1.4,
                'winter': 0.6,
                'spring': 1.0,
                'fall': 0.8
            }[season]
            
            predicted_generation = base_generation * season_mult
            predicted_usage = base_usage
            
            # Calculate if household will be buyer or seller
            delta = predicted_generation - predicted_usage
            trading_position = 'Seller' if delta > 0 else 'Buyer'
            
            # Calculate dynamic price based on time of day (using peak hour 14:00)
            market_price = engine.calculate_dynamic_price(
                total_supply=predicted_generation,
                total_demand=predicted_usage,
                time_of_day=14,
                season=season
            )
            
            future_predictions.append({
                'Date': future_date,
                'HouseholdID': household,
                'PredictedUsage': round(predicted_usage, 2),
                'PredictedGeneration': round(predicted_generation, 2),
                'TradingPosition': trading_position,
                'DynamicPrice': round(market_price, 4)
            })
    
    return pd.DataFrame(future_predictions)

# Generate predictions for next 10 weeks
future_df = predict_future_weeks(df, household_models)

# Display predictions grouped by date
print("\nTrading Forecast for Next 10 Weeks:")
# print(future_df['TradingPosition'].value_counts())
print(future_df)



Trading Forecast for Next 10 Weeks:
         Date HouseholdID  PredictedUsage  PredictedGeneration  \
0  2025-02-09          H1          945.15               342.88   
1  2025-02-16          H1          945.15               342.88   
2  2025-02-23          H1          945.15               342.88   
3  2025-03-02          H1          945.15               571.46   
4  2025-03-09          H1          945.15               571.46   
5  2025-03-16          H1          945.15               571.46   
6  2025-03-23          H1          945.15               571.46   
7  2025-03-30          H1          945.15               571.46   
8  2025-04-06          H1          945.15               571.46   
9  2025-04-13          H1          945.15               571.46   
10 2025-02-09          H2         1435.22               452.93   
11 2025-02-16          H2         1435.22               452.93   
12 2025-02-23          H2         1435.22               452.93   
13 2025-03-02          H2         1435.

In [11]:
future_df

,Date,HouseholdID,PredictedUsage,PredictedGeneration,TradingPosition,DynamicPrice
0,2025-02-09,H1,945.15,342.88,Buyer,0.4517
1,2025-02-16,H1,945.15,342.88,Buyer,0.5441
2,2025-02-23,H1,945.15,342.88,Buyer,0.4759
3,2025-03-02,H1,945.15,571.46,Buyer,0.3020
4,2025-03-09,H1,945.15,571.46,Buyer,0.2781
5,2025-03-16,H1,945.15,571.46,Buyer,0.2603
6,2025-03-23,H1,945.15,571.46,Buyer,0.2858
7,2025-03-30,H1,945.15,571.46,Buyer,0.2701
8,2025-04-06,H1,945.15,571.46,Buyer,0.3008
9,2025-04-13,H1,945.15,571.46,Buyer,0.2692


In [12]:
def predict_future_weeks(df, household_models, num_weeks=10):
    """Predict energy patterns and trading positions for next n weeks"""
    last_date = pd.to_datetime(df['Date'].max())
    future_dates = [last_date + pd.Timedelta(weeks=i+1) for i in range(num_weeks)]
    
    # First, calculate weekly market prices
    weekly_prices = {}
    for future_date in future_dates:
        season = get_season(future_date)
        
        # Calculate total supply and demand for the week
        total_supply = df.groupby('Date')['EnergyGeneratedFromRenewableSources'].mean().mean()
        total_demand = df.groupby('Date')['EnergyUsed'].mean().mean()
        
        # Apply seasonal adjustments
        season_mult = {
            'summer': 1.4,
            'winter': 0.6,
            'spring': 1.0,
            'fall': 0.8
        }[season]
        
        # Calculate centralized market price for the week
        market_price = engine.calculate_dynamic_price(
            total_supply=total_supply * season_mult,
            total_demand=total_demand,
            time_of_day=14,  # Using peak hour for base price
            season=season
        )
        weekly_prices[future_date] = market_price
    
    future_predictions = []
    for household in df['HouseholdID'].unique():
        # Get last 4 weeks of data for prediction
        recent_data = df[df['HouseholdID'] == household]
        
        # Use recent averages for baseline predictions
        base_usage = recent_data['EnergyUsed'].mean()
        base_generation = recent_data['EnergyGeneratedFromRenewableSources'].mean()
        
        for future_date in future_dates:
            season = get_season(future_date)
            season_mult = {
                'summer': 1.4,
                'winter': 0.6,
                'spring': 1.0,
                'fall': 0.8
            }[season]
            
            predicted_generation = base_generation * season_mult
            predicted_usage = base_usage
            
            # Calculate if household will be buyer or seller
            delta = predicted_generation - predicted_usage
            trading_position = 'Seller' if delta > 0 else 'Buyer'
            
            future_predictions.append({
                'Date': future_date,
                'HouseholdID': household,
                'PredictedUsage': round(predicted_usage, 2),
                'PredictedGeneration': round(predicted_generation, 2),
                'TradingPosition': trading_position,
                'DynamicPrice': round(weekly_prices[future_date], 4)
            })
    
    return pd.DataFrame(future_predictions)
# Generate predictions for next 10 weeks
future_df = predict_future_weeks(df, household_models)

# Display predictions grouped by date
print("\nTrading Forecast for Next 10 Weeks:")
print(future_df['TradingPosition'].value_counts())



Trading Forecast for Next 10 Weeks:
TradingPosition
Buyer     36
Seller    14
Name: count, dtype: int64


In [27]:
!pip install ace_tools

In [24]:
buyer

,Date,HouseholdID,PredictedUsage,PredictedGeneration,TradingPosition,DynamicPrice
7,2025-03-30,H1,945.15,571.46,Buyer,0.2601
17,2025-03-30,H2,1435.22,754.89,Buyer,0.2601
37,2025-03-30,H4,1206.54,653.16,Buyer,0.2601


In [71]:
sampleDf = future_df['Date'].unique()
sampleWeek = future_df[future_df['Date'] == sampleDf[7]]
sampleWeek['SP'] = [0.20, 0.40, 0.10, 0.30, 0.05]
sampleWeek['BP'] = [0.15, 0.30, 0.05, 0.25, 0.10]
buyer = sampleWeek[sampleWeek['TradingPosition'] == 'Buyer']
seller = sampleWeek[sampleWeek['TradingPosition'] == 'Seller']
buyer['RemainingDemand'] = buyer['PredictedUsage']-buyer['PredictedGeneration']
seller['RemainingSupply'] = seller['PredictedGeneration']-seller['PredictedUsage']
buyer.drop(columns=['PredictedUsage', 'PredictedGeneration', 'TradingPosition'], inplace=True)
seller.drop(columns=['PredictedUsage', 'PredictedGeneration', 'TradingPosition'], inplace=True)
seller['RemainingSupply'] += 800

C:\Users\utkar\AppData\Local\Temp\ipykernel_10780\2949590779.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\utkar\AppData\Local\Temp\ipykernel_10780\2949590779.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\utkar\AppData\Local\Temp\ipykernel_10780\2949590779.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [68]:
buyer

,Date,HouseholdID,DynamicPrice,SP,BP,RemainingDemand
7,2025-03-30,H1,0.2601,0.2,0.15,373.69
17,2025-03-30,H2,0.2601,0.4,0.30,680.33
37,2025-03-30,H4,0.2601,0.3,0.25,553.38


In [72]:
seller

,Date,HouseholdID,DynamicPrice,SP,BP,RemainingSupply
27,2025-03-30,H3,0.2601,0.10,0.05,927.63
47,2025-03-30,H5,0.2601,0.05,0.10,896.29


In [74]:
import pandas as pd

def optimal_trade_plan(buyers, sellers, trade_plan=None):
    if trade_plan is None:
        trade_plan = []
    
    # If all buyers' demands are met or all sellers' supplies are exhausted, return the trade plan
    if buyers['RemainingDemand'].sum() == 0 or sellers['RemainingSupply'].sum() == 0:
        return trade_plan, sum(t['Total Cost ($)'] for t in trade_plan)

    optimal_plan = None
    optimal_cost = float('inf')

    # Iterate over all possible buyer-seller pairs
    for buyer_idx, buyer in buyers[buyers['RemainingDemand'] > 0].iterrows():
        for seller_idx, seller in sellers[sellers['RemainingSupply'] > 0].iterrows():
            if buyer['BP'] >= seller['SP']:  # Check if a deal is possible
                trade_volume = min(buyer['RemainingDemand'], seller['RemainingSupply'])
                trade_price = (buyer['BP'] + seller['SP']) / 2  # Midpoint price
                
                # Create a new trade plan by adding this transaction
                new_trade = {
                    'Buyer': buyer['HouseholdID'],
                    'Seller': seller['HouseholdID'],
                    'Volume (kWh)': trade_volume,
                    'Price ($/kWh)': trade_price,
                    'Total Cost ($)': trade_volume * trade_price
                }
                
                # Update buyers and sellers
                new_buyers = buyers.copy()
                new_sellers = sellers.copy()
                new_buyers.at[buyer_idx, 'RemainingDemand'] -= trade_volume
                new_sellers.at[seller_idx, 'RemainingSupply'] -= trade_volume

                # Recursively find the optimal plan for the remaining buyers and sellers
                current_plan, current_cost = optimal_trade_plan(new_buyers, new_sellers, trade_plan + [new_trade])

                # Update the optimal plan if the current plan is better
                if current_cost < optimal_cost:
                    optimal_plan = current_plan
                    optimal_cost = current_cost
                print(new_buyers)
                print(new_sellers)
                

    return optimal_plan, optimal_cost

# Prepare example weekly data with columns 'HouseholdID', 'BP', 'SP', 'RemainingDemand', 'RemainingSupply'
# Buyers have 'RemainingDemand' > 0 and Sellers have 'RemainingSupply' > 0
buyers = buyer
sellers = seller

# Call the function with initial buyers and sellers
optimal_plan, total_cost = optimal_trade_plan(buyers, sellers)

# Display the optimal trade plan and total cost
optimal_plan_df = pd.DataFrame(optimal_plan)
print("Optimal Trade Plan:")
print(optimal_plan_df)

print("\nTotal Cost for Buyers:")
print(total_cost)

         Date HouseholdID  DynamicPrice   SP    BP  RemainingDemand
7  2025-03-30          H1        0.2601  0.2  0.15              0.0
17 2025-03-30          H2        0.2601  0.4  0.30              0.0
37 2025-03-30          H4        0.2601  0.3  0.25              0.0
         Date HouseholdID  DynamicPrice    SP    BP  RemainingSupply
27 2025-03-30          H3        0.2601  0.10  0.05             0.00
47 2025-03-30          H5        0.2601  0.05  0.10           216.52
         Date HouseholdID  DynamicPrice   SP    BP  RemainingDemand
7  2025-03-30          H1        0.2601  0.2  0.15             0.00
17 2025-03-30          H2        0.2601  0.4  0.30             0.00
37 2025-03-30          H4        0.2601  0.3  0.25           553.38
         Date HouseholdID  DynamicPrice    SP    BP  RemainingSupply
27 2025-03-30          H3        0.2601  0.10  0.05              0.0
47 2025-03-30          H5        0.2601  0.05  0.10            769.9
         Date HouseholdID  DynamicPrice   